In [1]:
import pandas as pd
import re
import os

"""

refseq - ncbi-ftp/gene/DATA/gene2refseq.gz
pubmed - ncbi-ftp/gene/DATA/gene2pubmed.gz
ensembl - ncbi-ftp/gene/DATA/gene2ensembl.gz
human - ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz
mart - useast.ensembl.org/index.html
neighbors - ncbi-ftp/gene/DATA/gene_neighbors.gz
orthologs - ncbi-ftp/gene/DATA/gene_orthologs.gz

"""

refseq = pd.read_table('gene2refseq')
pubmed = pd.read_table('gene2pubmed')
ensembl = pd.read_table('gene2ensembl')
mart = pd.read_csv('mart_export.txt',delimiter = ',')
neighbors = pd.read_table('gene_neighbors')
orthologs = pd.read_table('gene_orthologs')


C:\Users\Brendan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,6,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Brendan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [77]:
human = pd.read_table('Homo_sapiens.gene_info')
aliases = human['Synonyms'].tolist()
aliases_sep = []
for i in range(len(aliases)):
    my_list = aliases[i].split("|")
    my_list.append(human['Symbol'].iloc[i])
    aliases_sep.append(my_list)
human['Aliases'] = aliases_sep
refseq

,#tax_id,GeneID,status,RNA_nucleotide_accession.version,RNA_nucleotide_gi,protein_accession.version,protein_gi,genomic_nucleotide_accession.version,genomic_nucleotide_gi,start_position_on_the_genomic_accession,end_position_on_the_genomic_accession,orientation,assembly,mature_peptide_accession.version,mature_peptide_gi,Symbol
0,9,1246500,PROVISIONAL,-,-,NP_047184.1,10954455,NC_001911.1,10954454,348,1190,-,-,-,-,repA1
1,9,1246501,PROVISIONAL,-,-,NP_047186.1,10954457,NC_001911.1,10954454,2157,2912,+,-,-,-,repA2
2,9,1246502,VALIDATED,-,-,NP_047187.1,10954458,NC_001911.1,10954454,3040,4590,+,-,-,-,leuA
3,9,1246503,PROVISIONAL,-,-,NP_047188.1,10954459,NC_001911.1,10954454,4623,5714,+,-,-,-,leuB
4,9,1246504,VALIDATED,-,-,NP_047189.1,10954460,NC_001911.1,10954454,5717,7117,+,-,-,-,leuC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40141439,2749911,36959328,NaN,-,-,-,-,NC_038002.1,1418587958,119755,119828,-,-,-,-,trnP-GGG
40141440,2749911,36959329,PROVISIONAL,-,-,YP_009496746.1,1418587959,NC_038002.1,1418587958,116,2374,+,-,-,-,psaA
40141441,2749911,36959330,PROVISIONAL,-,-,YP_009496877.1,1418588090,NC_038002.1,1418587958,120098,120337,-,-,-,-,acpP
40141442,2749911,36959331,PROVISIONAL,-,-,YP_009496787.1,1418588000,NC_038002.1,1418587958,40018,40134,+,-,-,-,psbI


In [6]:
def compare_sets(list_data):
    for ele in list_data:
        col_names = ele.columns
        print(col_names)
data_list = refseq,pubmed,ensembl,human,mart,neighbors,orthologs
# compare_sets(data_list)

In [97]:
def combined_translate(input_file, file_type, trans_list):
    if (file_type == 'csv'):
        df = pd.read_csv(input_file)
    if (file_type == 'excel'):
        df = pd.read_excel(input_file)
    if (file_type == 'other'):
        df = pd.read_table(input_file)
    
    for item in trans_list:
        test_col = item[0]
        reference_df = item[1]
        reference_col = item[2]
        output_col = item[3]
        
        
        start_param = df[test_col].tolist()
        in_list = reference_df[reference_col].tolist()
        out_listoflist = []

        for i in range(len(start_param)):
            param = start_param[i]

            match_list=[]
            for j in range(len(in_list)):
                if str(param) in str(in_list[j]):
                    index_match = reference_df.iloc[j]
                    match_list.append(index_match)

            out_list = []
            for k in range(len(match_list)):
                if (output_col == 'MATCH'):
                    out_list.append(match_list[k][reference_col])
                else: 
                    out_list.append(match_list[k][output_col])

            out_listoflist.append(out_list)
        out_colname = 'output ' + reference_col
        df[out_colname] = out_listoflist

    
        nonmatches_ind = [ind for ind, x in enumerate(df[out_colname]) if len(x)==0 or x != x or x=='-']
        matches_ind = [ind for ind, x in enumerate(df[out_colname]) if len(x)!=0 or x == x or x!='-']
        nonmatches = df.iloc[nonmatches_ind]
        matches = df.iloc[matches_ind]
        
        return str(round(100*len(nonmatches)/len(df),4))+' % mismatch '
#         print(round(100*len(nonmatches)/len(df),4)," percent mismatch, ",input_file, 'to')
    
#     print(df)
    
        
          

In [82]:
### Name of input data file
input_file = 'md_2018_05_18_zhu_md-d-17-05991_sdc1.xlsx'

### Type of file (excel, csv, or other)
file_type = 'excel'

### translation 1 test col name
test_col1 = 'Gene Symbol'

### translation 1 reference dataset (human,gene,mart)
ref_df1 = human

### translation 1 reference col name
ref_col1 = 'Aliases'
    
### translation 1 output col name
out_col1 = 'MATCH'

### translation 1 use only if one translation, otherwise delete
trans_list = [[test_col1,ref_df1,ref_col1,out_col1]]
print(combined_translate(input_file,file_type,trans_list))



input_file = 'md_2018_05_18_zhu_md-d-17-05991_sdc1.xlsx'
file_type = 'excel'
test_col1 = 'Entrez Gene'
ref_df1 = human
ref_col1 = 'GeneID'
out_col1 = 'MATCH'
trans_list = [[test_col1,ref_df1,ref_col1,out_col1]]
print(combined_translate(input_file,file_type,trans_list))



input_file = 'RA_loci.txt'
file_type = 'csv'
test_col1 = 'Gene'
ref_df1 = human
ref_col1 = 'Aliases'
out_col1 = 'MATCH'
trans_list = [[test_col1,ref_df1,ref_col1,out_col1]]
print(combined_translate(input_file,file_type,trans_list))



input_file = 'Yamamoto.txt'
file_type = 'other'
test_col1 = 'Gene'
ref_df1 = human
ref_col1 = 'Aliases'
out_col1 = 'MATCH'
trans_list = [[test_col1,ref_df1,ref_col1,out_col1]]
print(combined_translate(input_file,file_type,trans_list))



input_file = 'RADB.txt'
file_type = 'other'
test_col1 = 'Gene'
ref_df1 = human
ref_col1 = 'Aliases'
out_col1 = 'MATCH'
trans_list = [[test_col1,ref_df1,ref_col1,out_col1]]
print(combined_translate(input_file,file_type,trans_list))


13.099 % mismatch md_2018_05_18_zhu_md-d-17-05991_sdc1.xlsx to Aliases
12.7796 % mismatch md_2018_05_18_zhu_md-d-17-05991_sdc1.xlsx to GeneID
7.5472 % mismatch RA_loci.txt to Aliases
10.4762 % mismatch Yamamoto.txt to Aliases
19.0476 % mismatch RADB.txt to Aliases


In [98]:
def match_lists(data_list):
    for i in range(len(data_list)):
        for j in range(len(data_list)):
            if (i != j):
                ### translation 1 test col name
                test_col1 = data_list[i][1]

                ### translation 1 reference dataset (human,gene,mart)
                if (data_list[j][2] == 'csv'):
                    ref_df1 = pd.read_csv(data_list[j][0])
                if (data_list[j][2] == 'excel'):
                    ref_df1 = pd.read_excel(data_list[j][0])
                if (data_list[j][2] == 'other'):
                    ref_df1 = pd.read_table(data_list[j][0])

                ### translation 1 reference col name
                ref_col1 = data_list[j][1]

                ### translation 1 output col name
                out_col1 = 'MATCH'
                
                trans_list = [[test_col1,ref_df1,ref_col1,out_col1]]
                match_percent = combined_translate(data_list[i][0], data_list[i][2], trans_list)
                print(match_percent," data_list[i][0], 'to', data_list[j][0])
#                 print(match_percent)

SyntaxError: EOL while scanning string literal (<ipython-input-98-f263c23531b2>, line 24)

In [99]:
md_2018 = pd.read_excel('md_2018.xlsx')
# https://journals.lww.com/md-journal/Fulltext/2018/06010/Identification_of_key_genes_in_rheumatoid.86.aspx
# A total of 313 genes (DEGs) were identified to be differentially expressed between RA and NC samples

ra_loci = pd.read_csv('RA_loci.txt')
# https://academic.oup.com/view-large/27924154
# Genetic loci associated with susceptibility to RA

yamamoto = pd.read_table('Yamamoto.txt')
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4729856/
# RA susceptible genes

radb = pd.read_table('RADB.txt')
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4164886/
# Genes and genetic regions that have the strongest association with RA susceptibility. 

okada = pd.read_table('okada.txt')
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3944098/#SD2
# Novel rheumatoid arthritis risk loci identified by trans-ethnic GWAS meta-analysis in >100,000 subjects.


l1 = ['md_2018.xlsx', 'Gene Symbol','excel']
l2 = ['ra_loci.txt','Gene','csv']
l3 = ['Yamamoto.txt','Gene','other']
l4 = ['RADB.txt','Gene','other']
l5 = ['okada.txt','Gene','other']

match_lists([l1,l2,l3,l4,l5])



96.8051 % mismatch   percent mismatch,  md_2018.xlsx to ra_loci.txt
96.8051 % mismatch   percent mismatch,  md_2018.xlsx to Yamamoto.txt
99.0415 % mismatch   percent mismatch,  md_2018.xlsx to RADB.txt
96.8051 % mismatch   percent mismatch,  md_2018.xlsx to okada.txt
90.566 % mismatch   percent mismatch,  ra_loci.txt to md_2018.xlsx
15.0943 % mismatch   percent mismatch,  ra_loci.txt to Yamamoto.txt
76.4151 % mismatch   percent mismatch,  ra_loci.txt to RADB.txt
14.1509 % mismatch   percent mismatch,  ra_loci.txt to okada.txt
92.381 % mismatch   percent mismatch,  Yamamoto.txt to md_2018.xlsx
19.0476 % mismatch   percent mismatch,  Yamamoto.txt to ra_loci.txt
80.9524 % mismatch   percent mismatch,  Yamamoto.txt to RADB.txt
9.5238 % mismatch   percent mismatch,  Yamamoto.txt to okada.txt
95.2381 % mismatch   percent mismatch,  RADB.txt to md_2018.xlsx
19.0476 % mismatch   percent mismatch,  RADB.txt to ra_loci.txt
14.2857 % mismatch   percent mismatch,  RADB.txt to Yamamoto.txt
14.2857 